# Exercise 17

## Analyze how travelers expressed their feelings on Twitter

A sentiment analysis job about the problems of each major U.S. airline. 
Twitter data was scraped from February of 2015 and contributors were 
asked to first classify positive, negative, and neutral tweets, followed
by categorizing negative reasons (such as "late flight" or "rude service").

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# read the data and set the datetime as the index
tweets = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Tweets.zip', index_col=0)

tweets.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
tweet_id,,,,,,,,,,,,,,
570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


- **Variables Relevantes**
- **airline_sentiment** Variable a explicar 'y' a analizar, 3 categorias (neutral, positive, negative)'/n'
- **airline** Variable aerolinea - Variable explicativa 'x1', 6 categorias (United, US airways, American, Southwest, Delta, Virgin America)
- **text** Variable de texto a analizar-variable explicativa 'x2'
- **user_timezone** Zona horaria del tweet.

In [ ]:
tweets.user_timezone.isna().sum()

In [ ]:
tweets.shape

### Proportion of tweets with each sentiment

In [ ]:
tweets['airline_sentiment'].value_counts()

In [ ]:
tweets.airline_sentiment.isna().sum()

In [ ]:
tweets.text.isna().sum()

In [ ]:
tweets.airline.isna().sum()

### Proportion of tweets per airline


In [ ]:
tweets['airline'].value_counts()

In [ ]:
pd.Series(tweets["airline"]).value_counts().plot(kind = "bar",figsize=(8,6),rot = 0)

In [ ]:
pd.crosstab(index = tweets["airline"],columns = tweets["airline_sentiment"]).plot(kind='bar',figsize=(10, 6),alpha=0.5,rot=0,stacked=True,title="Sentiment by airline")

# Exercise 17.1 

Predict the sentiment using CountVectorizer

use Random Forest classifier

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [ ]:
# Use CountVectorizer to create document-term matrices from X : 
vect = CountVectorizer(lowercase=False)
X_dtm = vect.fit_transform(X)

In [ ]:
X_dtm.shape

In [ ]:
Col= list(vect.vocabulary_.keys())

In [ ]:
temp1=X_dtm.todense()

In [ ]:
temp= pd.DataFrame(data=temp1, columns=Col)

In [ ]:
temp.shape

In [ ]:
temp.index

In [2]:
airline = pd.DataFrame(data= tweets.airline)

In [ ]:
airline.shape

In [ ]:
airline.index

In [ ]:
temp.index

In [3]:
airline.reset_index(drop=True,inplace=True)

In [ ]:
airline.index

In [4]:
airline1=pd.get_dummies(airline)

In [ ]:
airline1

In [5]:
airline1.drop('airline_American',axis=1,inplace=True)

In [ ]:
X= pd.concat([temp, airline1], axis=1)

In [ ]:
X.head()

In [ ]:
# Definición Random forest Clasiffier
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, n_jobs=-1)
rnd_clf.fit(X,y)

In [ ]:
# probando el algoritmo mediante Cross-Validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rnd_clf, X, y, cv=10)

In [ ]:
 print('CV accuracy scores: %s' % scores)

In [ ]:
 print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
np.std(scores))) 

# Exercise 17.2 

Remove stopwords, then predict the sentiment using CountVectorizer.

use Random Forest classifier

In [ ]:
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [ ]:
# Use CountVectorizer to create document-term matrices from X : 
vect1 = CountVectorizer(lowercase=False, stop_words='english')
X_dtm1 = vect1.fit_transform(X)
Col= list(vect1.vocabulary_.keys())

In [ ]:
temp1=X_dtm1.todense()

In [ ]:
temp1= pd.DataFrame(data=temp1, columns=Col)

In [ ]:
temp1.shape

In [ ]:
X1= pd.concat([temp1, airline1], axis=1)
X1.head()

In [ ]:
X1.shape

In [ ]:
# probando el algoritmo mediante Cross-Validation
from sklearn.model_selection import cross_val_score
scores1 = cross_val_score(rnd_clf, X1, y, cv=10)

In [ ]:
 print('CV accuracy scores: %s' % scores1)

In [ ]:
 print('CV accuracy: %.5f +/- %.3f' % (np.mean(scores),
np.std(scores))) 

# Exercise 17.3

Increase n_grams size (with and without stopwords),  then predict the sentiment using CountVectorizer

use Random Forest classifier

## Predicción de Sentimientos con "Stopwords"

In [6]:
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [9]:
vect2 = CountVectorizer(ngram_range=(1, 2), lowercase=False, stop_words='english',max_features=10000)
X_dtm2 = vect2.fit_transform(X)
Col= list(vect2.vocabulary_.keys())
temp2=X_dtm2.todense()

In [10]:
temp2= pd.DataFrame(data=temp2, columns=Col)

In [11]:
temp2.shape

(14640, 10000)

In [13]:
# Definición Random forest Clasiffier
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, n_jobs=-1)
# probando el algoritmo mediante Cross-Validation
from sklearn.model_selection import cross_val_score
scores2 = cross_val_score(rnd_clf, temp2, y, cv=10)

In [15]:
 print('CV accuracy: %.5f +/- %.3f' % (np.mean(scores2),
np.std(scores2))) 

CV accuracy: 0.62486 +/- 0.006


## Predicción de Sentimientos sin "Stopwords"

In [16]:
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})
vect3 = CountVectorizer(ngram_range=(1, 2), lowercase=False,min_df=3,max_features=10000)
X_dtm3 = vect3.fit_transform(X)
Col= list(vect3.vocabulary_.keys())
temp3=X_dtm3.todense()
temp3= pd.DataFrame(data=temp3, columns=Col)
temp3.shape

(14640, 10000)

In [17]:
X3= pd.concat([temp3, airline1], axis=1)
X3.head()

,VirginAmerica,What,said,plus,you,ve,added,commercials,to,the,...,800 433,433 7300,5350,flight 3056,understaffing,airline_Delta,airline_Southwest,airline_US Airways,airline_United,airline_Virgin America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
# Definición Random forest Clasiffier
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, n_jobs=-1)
# probando el algoritmo mediante Cross-Validation
from sklearn.model_selection import cross_val_score
scores3 = cross_val_score(rnd_clf, X3, y, cv=10)

In [19]:
 print('CV accuracy: %.5f +/- %.3f' % (np.mean(scores3),
np.std(scores3))) 

CV accuracy: 0.62493 +/- 0.006


# Exercise 17.4

Predict the sentiment using TfidfVectorizer.

use Random Forest classifier

In [20]:
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':-1,'neutral':0,'positive':1})

In [21]:
# create a document-term matrix using TF-IDF
vect = TfidfVectorizer(stop_words='english')
dtm = vect.fit_transform(X)
features = vect.get_feature_names()
dtm.shape

(14640, 14770)

In [25]:
# Definición Random forest Clasiffier
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, n_jobs=-1)
# probando el algoritmo mediante Cross-Validation
rnd_clf.fit(dtm,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
# Definición Random forest Clasiffier
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=10, n_jobs=-1)
# probando el algoritmo mediante Cross-Validation
from sklearn.model_selection import cross_val_score
scores3 = cross_val_score(rnd_clf, dtm, y, cv=10)

In [29]:
 print('CV accuracy: %.5f +/- %.3f' % (np.mean(scores3),
np.std(scores3))) 

CV accuracy: 0.62691 +/- 0.000
